<center>
    
# X Solana tweets Analysis

</center>

We are going to use official twitter API to pull out Solana related posts, and then analyse them using roBERTa model and OpenAI gpt4-o model. Let's start with basic imports:

In [60]:
import tweepy
import pandas as pd
from langchain_community.callbacks import get_openai_callback
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

**Warning:** Free tier of twitter API only allows pulling **100 tweets** a month, hence after the inital run we save out pulled tweets to a pickle file, for reruns and analysis. 

So, to run it freshly, remove the commenter API call and load data from API rather than a dataframe.
Assumption is for actual project we won't have this bottleneck, worst case we can make use of available open source library for pulling tweets, however these are unreliable and risk breakage. 

In [93]:
# # Set up Twitter API credentials
# BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAECv0AEAAAAAs9DtkGAVpcg0AhI6lIrhmDE350Q%3D3eGGcicaf6hxSdjkU600ndNoMxjAPZtpEQjlb2BbvQU3bFODbm"

# # Authenticate and connect
# client = tweepy.Client(bearer_token=BEARER_TOKEN)

# # Fetch tweets from the past 7 days
# query = "Solana -is:retweet lang:en"
# tweets = client.search_recent_tweets(query=query, max_results=10, tweet_fields=["created_at", "text", "public_metrics"])

# # Store data in a DataFrame
# data = [{"time": t.created_at, "text": t.text, "likes": t.public_metrics["like_count"]} for t in tweets.data]
# df = pd.DataFrame(data)
# df.to_pickle('solana_tweets')

df = pd.read_pickle('solana_tweets')
df.head()


,time,text,likes
0,2025-03-27 09:55:17+00:00,"PLUME Coin Price Prediction, What's Next for P...",0
1,2025-03-27 09:55:12+00:00,🔥@GamerBoom_ Airdrop-$11M Backed by Solana &am...,0
2,2025-03-27 09:55:09+00:00,"@ZygoFrog Good Job Team Zygo , base and Solana...",0
3,2025-03-27 09:55:08+00:00,"Buy the dip? Bro, I’ve been buying dips since ...",0
4,2025-03-27 09:55:07+00:00,🔥@GamerBoom_ Airdrop-$11M Backed by Solana &am...,0


In [95]:
# Print number of rows and also print the tweets in full length for visuaalization
print("No. of rows: ", df.shape[0])
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(df)

No. of rows:  10


,time,text,likes
0,2025-03-27 09:55:17+00:00,"PLUME Coin Price Prediction, What's Next for PLUME coin: Down to 15.8 Cents? https://t.co/Q8xtyZ8Khr PLUME Coin Price Prediction, Analysis, News and Update -27 March 2025 #cryptoanalysis #pricepredictions #solana \n #cryptocop #trading #trading #plume \nGet the latest Plume c…",0
1,2025-03-27 09:55:12+00:00,🔥@GamerBoom_ Airdrop-$11M Backed by Solana &amp; NVIDIA\n\n1️⃣ Sign up: https://t.co/o5f9qayWfS \n2️⃣ Do social tasks + daily check-in \n3️⃣ Invite friends or download the app to get more pts​\n@CryptoSaiV2 @CryptOguuz @adilk0870 @benson_btc @mominsaqib,0
2,2025-03-27 09:55:09+00:00,"@ZygoFrog Good Job Team Zygo , base and Solana to the 🌙 🐸🐸🐸",0
3,2025-03-27 09:55:08+00:00,"Buy the dip? Bro, I’ve been buying dips since BTC was $40K. Just send it already. 😂\n \n https://t.co/OCqmHMKOgC solana,memecoın,BITCOIN,Crypto $QQQ $GME $VKTX $FFIE $AMC $GWAV $CRKN $SPY $GNS $HOLO $NVDA $CYN $BSFC",0
4,2025-03-27 09:55:07+00:00,🔥@GamerBoom_ Airdrop-$11M Backed by Solana &amp; NVIDIA\n\n1️⃣ Sign up: https://t.co/MXIrWbifgZ \n2️⃣ Do social tasks + daily check-in \n3️⃣ Invite friends or download the app to get more pts​\n@beautyofhelin @isadora288881 @MartiniGuyYT @noroichan_game @TheDustyBC,0
5,2025-03-27 09:55:07+00:00,The market is always searching for the best price. Find the hidden gems before the herd does. @JupiterExchange is building something special. 💚 #Solana #JupiterExchange #JUPtoken #Jupnet,0
6,2025-03-27 09:55:04+00:00,@Solana_Slot Gm boss 💎,0
7,2025-03-27 09:55:02+00:00,🔥@GamerBoom_ Airdrop-$11M Backed by Solana &amp; NVIDIA\n\n1️⃣ Sign up: https://t.co/eNejz56srf \n2️⃣ Do social tasks + daily check-in \n3️⃣ Invite friends or download the app to get more pts​\n@dyor_100x @zenryoku_eth @gg776650 @golden_kripto @blokinfluencer,0
8,2025-03-27 09:55:00+00:00,Every wallet gets 250k $KING 🪂\n\nDrop your Solana $Sol Address &amp; RT ♻️\n\nLike &amp; Rt pinned post 📌 https://t.co/hhyzCo9iE3,0
9,2025-03-27 09:55:00+00:00,"@solana_zzz Whoa, that’s some wild code! If you’re into unique vibes, check out Catapult Corgi—we just dropped on Solana! Transparent community, no gimmicks. Join us\n@catapultcorgi\nwith $CCC #memecoin!",0


## Analysis
Now that we have our tweet, we now analyze our tweets for sentiment, credibility, relevance, reliability and category. We plan to use:

- roBERTa library for sentiment analysis(state of the art for sentiment analysis, and indiviually suited for the task rather than general llm)
- openAI gpt 4-0 model for rest of analysis. This assigns a score between 1-5 and also reasoning for the score.

### Warning: I have used my own personal GPT which is costly, hence the access key has been commented out to avoid unfair usage. 

In [74]:
import torch
import json
from ollama import Client
from transformers import pipeline

# Initialize sentiment analyzer with roBERTa
sentiment_analyzer = pipeline(
    "text-classification", 
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device=0 if torch.cuda.is_available() else -1
)

# Define label mapping for the sentiment model
label_mapping = {
    "LABEL_0": "NEGATIVE",
    "LABEL_1": "NEUTRAL",
    "LABEL_2": "POSITIVE"
}

# Apply sentiment analysis
def analyze_sentiment(text):
    result = sentiment_analyzer(text, truncation=True)[0]  # Single call
    return label_mapping[result["label"]], result["score"]

# Apply function and unpack into two columns
df[["sentiment", "sentiment_score"]] = df["text"].apply(lambda x: pd.Series(analyze_sentiment(x)))
# Convert timestamp column to string format (ISO 8601)
df["time"] = df["time"].astype(str)


# Prompt for the LLM for analysis, it can be easily modified to change the queries on a as-needed basis.
messages = [
    HumanMessage(
        content=[
            {
                "type": "text",
                "text": """
                    You MUST return a valid JSON object following this exact schema:
                    {
                      "credibility": "int between 1-5",
                      "credibility_reasoning": "string",
                      "relevance": "int between 1-5",
                      "relevance_reasoning": "string",
                      "reliability": "int between 1-5",
                      "reliability_reasoning": "string",
                      "category": "string",
                      "category_reasoning": "string"
                    }
                    Do NOT include extra text. Output only JSON string no indentation nothing.
                """
            },
            {
                "type": "text",
                "text": f"Text to analyze: {twt}"
            }
        ]
    )
]

chat = ChatOpenAI(model="gpt-4o", max_tokens=256, api_key = "YOUR_API_KEY")

json_array = []
for twt in df["text"]: 
    with get_openai_callback() as cb:
        result = chat.invoke(messages)
        print(cb)
        print("response: ", result.content)
    
    # Convert JSON string to Python list
        json_obj = json.loads(result.content)
        json_array.append(json_obj)
        
# Print the json and convert intro new df
print(json.dumps(json_array, indent=4))

new_df = pd.DataFrame(json_array)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


analyse: 
Tokens Used: 342
	Prompt Tokens: 193
		Prompt Tokens Cached: 0
	Completion Tokens: 149
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0019725
response:  {"credibility": "2", "credibility_reasoning": "The tweet is a mention of multiple Twitter handles followed by an informal expression 'ily sis' and a link. There is no context or information to assess credibility.", "relevance": "1", "relevance_reasoning": "The tweet does not provide any substantial information or context related to specific topics or events.", "reliability": "2", "reliability_reasoning": "The tweet does not contain any verifiable information or claims, making it not particularly reliable.", "category": "Social Media Interaction", "category_reasoning": "The tweet involves a common social media interaction with mentions and an informal message, typical of personal communication or networking."}
analyse: 
Tokens Used: 319
	Prompt Tokens: 193
		Prompt Tokens Cached: 0
	Completion Tokens: 126
		

In [97]:
# visualize the new df
new_df.head()

,credibility,credibility_reasoning,relevance,relevance_reasoning,reliability,reliability_reasoning,category,category_reasoning
0,2,The tweet is a mention of multiple Twitter han...,1,The tweet does not provide any substantial inf...,2,The tweet does not contain any verifiable info...,Social Media Interaction,The tweet involves a common social media inter...
1,2,The content is a social media post with multip...,1,The post lacks substantive information or rele...,2,"Given the informal nature and lack of content,...",Social Media Post,This is a typical social media interaction whe...
2,2,The post is a personal message with no factual...,1,The content is personal and lacks relevance to...,2,"The message includes a social media link, but ...",personal communication,The message is a personal greeting between ind...
3,2,The tweet consists of tagged usernames and a s...,1,The tweet lacks context or content that indica...,2,The reliability is uncertain as the tweet prov...,social media,The format of the tweet is typical for social ...
4,2,The text appears to be a personal message or c...,1,The text lacks specific content or context rel...,2,With no factual content to verify and its natu...,Social Media Interaction,The format and content suggest it is a social ...


In [99]:
# Combine sentiment analysis and llm analysis into single dataframe. 
new_df = pd.DataFrame(json_array)
df = pd.concat([df, new_df], axis=1)

# Saved data to csv
df.to_csv('tweetsAnalysis.csv')

## Conclusion and further improvements

In the above notebook we showed a POC to pull data related to Solana, and did analysis on the market sentiments through multiple angles such as sentiment, credibility, relevance, reliability. Further improvements possible:

1. Use paid account to get all tweets in the past 7 days and not be limited with free tier.
2. Custom train gpt on expected type of analysis/ output for better responses.
3. Clearly define what we mean by credibility, category etc for better tagging

### Check the pdf version for running details of the notebook.
### Check tweetsAnalysis.csv for results.